# Test the Trend Index

## Check if index is up!

In [17]:
import requests
res = requests.get('http://foodmap.isti.cnr.it:9200', auth=('elastic', 'changeme'))
print(res.content)

{
  "name" : "0PsTttc",
  "cluster_name" : "foodmap-cluster",
  "cluster_uuid" : "2YIbFkvuRqaPWhEjuEYnwA",
  "version" : {
    "number" : "5.4.0",
    "build_hash" : "780f8c4",
    "build_date" : "2017-04-28T17:43:27.229Z",
    "build_snapshot" : false,
    "lucene_version" : "6.5.0"
  },
  "tagline" : "You Know, for Search"
}



In [18]:
from elasticsearch import Elasticsearch
es = Elasticsearch(['foodmap.isti.cnr.it'], http_auth=('elastic', 'changeme'), port=9200)
print es

<Elasticsearch([{u'host': u'foodmap.isti.cnr.it'}])>


## Index operations

#### Delete index, delete tweets by id and query

In [7]:
es.indices.delete(index='trend', ignore=[400, 404])

{u'acknowledged': True}

In [ ]:
es.delete_by_query(index='trend', doc_type='tweet', body={
    'query': {
        "match_all" : {}
    }
})

#### Count elements in index

In [13]:
es.count(index='trend', doc_type='tweet')

{u'_shards': {u'failed': 0, u'successful': 5, u'total': 5}, u'count': 194978}

#### GET all docs from index

In [19]:
res = es.search(index="trend", doc_type='tweet', size=10, body={"query": {"match_all": {}}})
# [x['_source'] for x in res['hits']['hits']]
for r in res['hits']['hits'][:20]:
    print r
    print 

{u'_score': 1.0, u'_type': u'tweet', u'_id': u'840206621422768128', u'_source': {u'username': u'theicesmith', u'lang': u'en', u'text_categories': [u'ice cream'], u'city': None, u'text': u'Dinner party sharing pots #dinnerplans #dinnerparty #dinner #home #candles #icecream #sorbet https://t.co/frEJH6BrL7', u'img_categories': None, u'hashtags': [u'#dinnerplans', u'#dinnerparty', u'#dinner', u'#home', u'#candles', u'#icecream', u'#sorbet'], u'img_flag': True, u'timestamp_ms': u'1489155843488', u'bounding_box': None, u'coords': None, u'id_str': u'840206621422768128', u'country': u'united kingdom', u'id': 840206621422768128, u'created_at_day': 20170310, u'created_at_datetime': u'2017-03-10T14:24:03', u'media_url': None}, u'_index': u'trend'}

{u'_score': 1.0, u'_type': u'tweet', u'_id': u'840206630750904320', u'_source': {u'username': u'thesimpleparent', u'lang': u'en', u'text_categories': [u'pizza'], u'city': u'houston', u'text': u"RT @seeingdandy: The @IncrediblePizza Spin Coaster in St. 

#### GET specific tweets

In [ ]:
es.get(index='trend', doc_type='tweet', id=880404048834416644)

In [ ]:
results = es.search(index='trend', doc_type='tweet', size=100, body={
  'query': {
    'match': {
      'text_categories': 'donuts',
#       'day': 20170618,
#         'sort': [
#         {'committed_date': {'order': 'desc'}}
#       ],
     }
  }
})

print results
print len(results["hits"]["hits"])

#### TEST image classifier

In [ ]:
# test some of the urls from the indexed tweets
# by reclassifying it with the IMG classifier hosted on our machine
request_string = 'http://test.tripbuilder.isti.cnr.it:8080/FoodRecognition/services/IRServices/recognizeByURL?imgURL='
for tweet in results["hits"]["hits"]:
    img_url = tweet["_source"]["media_url"]
    res = requests.get(request_string + img_url)

    print tweet["_source"]["text"]
    print img_url
    print res.content
    print json.loads(res.text)["guessed"]
    print get_image_category(img_url)
    print 
    print

In [ ]:
# import requests
# import json

# response = requests.get('http://test.tripbuilder.isti.cnr.it:5000/sync')
# json_data = json.loads(response.text)
# json_data

### Index from file

In [ ]:
import json
import os
import sys
import time
os.chdir("/home/foodmap/food101/")
sys.path.append(os.getcwd())
from elasticsearch import Elasticsearch, RequestError
from processing.preprocess_tweet import process_tweet
from processing.twitter.Tweet import Tweet

In [ ]:
inputFile="/home/foodmap/data/food-tweets/food-tweets-2017-03-10.json.gz"
indexName="trend"

tweetsAsDict = Tweet.getTweetAsDictionary(inputFile)
i = 0
numIndex = 0
for tweet in tweetsAsDict:
    i += 1
    if i % 10 == 0:
        print "Processed tweets: ", i
        print "Indexed tweets: ", numIndex

    new_tweet = process_tweet(tweet, forStream=False)
    if new_tweet is None:
        continue

    try:
        if i > 1480:
            print new_tweet
        # added request_timeout to avoid elasticsearch.exceptions.ConnectionTimeout
        es.index(index=indexName, doc_type='tweet', id=new_tweet["id"], body=new_tweet)
        numIndex += 1
        # print "Indexed tweet: ", new_tweet["id"]
    except RequestError as e:
        print "Couldn't index tweet id: ", new_tweet["id"]
        print e.status_code, e.message
        time.sleep(60)

print "Processed tweets: ", i
print "Indexed tweets: ", numIndex